In [142]:
import json

# Load JSONs
with open("agent_results.json") as f:
    agent_data = json.load(f)
with open("qa/nvda_ground_truth3.json") as f:
    ground_truth = json.load(f)

In [143]:
agent_data

[{'question_number': 1,
  'question': 'Report the Gross Margin (%) over the last 5 quarters, with values.',
  'raw_response': '{\'input\': \'\\nYou are a financial analyst agent that can use tools.\\n\\nUse the retriever tool when you need to fetch financial data. You can only call this tool ONCE.\\nUse the comparison tool when you need to compute YoY or QoQ comparisons.\\nUse the calculator tool to compute or derive financial ratios or custom metrics (e.g., Opex Ã· Operating Income, Gross Margin %, Net Margin).\\n\\nOnce done, return\\n1. The **final structured JSON output** in this format:\\n2. **Prose explanation**, converting the JSON output into a formatted table\\n\\n{\\n"query": "...",\\n"data_values": [...],\\n"computed_values": [...],\\n"citations": [{"report": "...", "page": ..., "section": "..."}],\\n"tools": ["<list the tools you actually used>"],\\n"tools_count": <total number of tools used>\\n}\\n\\nGuidelines:\\n- `data_values` contain the raw financial figures, correspo

In [144]:
ground_truth

[{'query': 'Report the Gross Margin (%) over the last 5 quarters, with values.',
  'question_type': 'trend',
  'expected_data_values': [{'fiscal_year': '2025',
    'fiscal_quarter': 'Q2',
    'gross_margin': 75.1},
   {'fiscal_year': '2025', 'fiscal_quarter': 'Q3', 'gross_margin': 74.6},
   {'fiscal_year': '2025', 'fiscal_quarter': 'Q4', 'gross_margin': 73.0},
   {'fiscal_year': '2026', 'fiscal_quarter': 'Q1', 'gross_margin': 60.5},
   {'fiscal_year': '2026', 'fiscal_quarter': 'Q2', 'gross_margin': 72.4}],
  'expected_computed_values': [{'fiscal_year': '2025',
    'fiscal_quarter': 'Q2',
    'gross_margin': 75.1},
   {'fiscal_year': '2025', 'fiscal_quarter': 'Q3', 'gross_margin': 74.6},
   {'fiscal_year': '2025', 'fiscal_quarter': 'Q4', 'gross_margin': 73.0},
   {'fiscal_year': '2026', 'fiscal_quarter': 'Q1', 'gross_margin': 60.5},
   {'fiscal_year': '2026', 'fiscal_quarter': 'Q2', 'gross_margin': 72.4}],
  'expected_citations': [{'report': 'FY25Q2_QuarterlyPresentation',
    'year': '

In [145]:
import re

def parse_year_str(y):
    """Normalize any year format ('25','FY25','2025','FY2025') -> '2025' (string)."""
    y = str(y).strip()
    y = y.replace("FY", "").replace("fy", "")
    if len(y) == 2:  # e.g. '25' -> '2025' (assume 20xx)
        return f"20{y}"
    if len(y) == 4:
        return y
    # if empty or malformed, return None
    return None


def normalize_agent_data_generic(agent_list):
    normalized = {}
    for item in agent_list:
        key = None

        # --- derive key ---
        if "quarter" in item:
            # e.g. "Q2 FY26" or "FY26 Q2"
            qtext = item["quarter"].strip()
            parts = re.findall(r"(Q\d)\s*(FY)?(\d{2,4})", qtext)
            if parts:
                q, _, y = parts[0]
                year = parse_year_str(y)
                key = f"FY{year}-{q}"
        elif "year" in item:
            y = parse_year_str(item["year"])
            if y:
                key = f"FY{y}"
        elif "fiscal_year" in item:
            y = parse_year_str(item["fiscal_year"])
            key = f"FY{y}"

        if not key:
            continue

        # --- extract + map metrics ---
        entry = {}
        for k, v in item.items():
            k_norm = k.lower().strip()
            # numeric cleanup
            if isinstance(v, str) and v.endswith("%"):
                try:
                    v = float(v.replace("%", "").strip())
                except ValueError:
                    pass
            # field mapping
            if k_norm in ["operating_expenses", "opex"]:
                entry["opex"] = float(v)
            elif "gross" in k_norm and "margin" in k_norm:
                entry["gross_margin"] = float(v)
            elif k_norm in ["computed_value", "efficiency_ratio"]:
                entry["efficiency_ratio"] = float(v)
            elif "yoy" in k_norm:
                entry["yoy_change (%)"] = float(v) if v not in [None, "None", ""] else None
            else:
                entry[k_norm] = v

        normalized[key] = entry
    return normalized


def normalize_gt_data_generic(gt_list):
    normalized = {}
    for item in gt_list:
        key = None
        fy = item.get("fiscal_year") or item.get("year")
        if fy:
            year = parse_year_str(fy)
            if "fiscal_quarter" in item:
                q = item["fiscal_quarter"].strip()
                key = f"FY{year}-{q}"
            else:
                key = f"FY{year}"

        if not key:
            continue

        entry = {}
        for k, v in item.items():
            k_norm = k.lower().strip()
            if isinstance(v, str) and v.endswith("%"):
                v = float(v.replace("%", "").strip())

            if k_norm in ["operating_expenses", "opex"]:
                entry["opex"] = float(v)
            elif "gross" in k_norm and "margin" in k_norm:
                entry["gross_margin"] = float(v)
            elif k_norm in ["computed_value", "efficiency_ratio"]:
                entry["efficiency_ratio"] = float(v)
            elif "yoy" in k_norm:
                entry["yoy_change (%)"] = float(v) if v not in [None, "None", ""] else None
            else:
                entry[k_norm] = v
        normalized[key] = entry
    return normalized


def compare_financials(agent_list, gt_list, value_fields=None, tolerance=0):
    agent_norm = normalize_agent_data_generic(agent_list)
    gt_norm = normalize_gt_data_generic(gt_list)

    print("2) NORMALISED GT->Agent:::::::::::")
    print(gt_norm)
    print(agent_norm)
    print("::::::::::::::::")

    if value_fields is None:
        value_fields = ["opex", "gross_margin", "efficiency_ratio", "yoy_change (%)"]

    total_fields = 0   # <-- count all fields we try to compare
    correct = 0
    mismatches = []

    for key, gt_entry in gt_norm.items():
        if key in agent_norm:
            agent_entry = agent_norm[key]
            for field in value_fields:
                if field in gt_entry:
                    total_fields += 1  # <-- increment for every comparable GT field
                    if field in agent_entry:
                        try:
                            gt_val = float(gt_entry[field])
                            agent_val = float(agent_entry[field])
                            diff = abs(gt_val - agent_val)

                            if diff <= tolerance:
                                correct += 1
                            else:
                                mismatches.append({
                                    "key": key,
                                    "field": field,
                                    "expected": gt_val,
                                    "actual": agent_val,
                                    "difference": diff
                                })
                        except Exception:
                            pass
                    else:
                        mismatches.append({
                            "key": key,
                            "field": field,
                            "expected": gt_entry[field],
                            "actual": "field_missing_in_agent",
                            "difference": "no_match"
                        })
        else:
            for field in value_fields:
                if field in gt_entry:
                    total_fields += 1
            mismatches.append({
                "key": key,
                "field": "missing",
                "expected": gt_entry,
                "actual": "no_agent_value",
                "difference": "no_agent_value"
            })

    accuracy = (correct / total_fields * 100) if total_fields > 0 else 0
    return accuracy, mismatches


In [146]:
# def extract_json_and_prose(response_text: str):
#     """
#     Extracts JSON (fenced or unfenced) and prose explanation.
#     Captures the full JSON, even with nested braces.
#     """
#     # Match fenced or unfenced JSON
#     pattern = re.compile(
#         r"```(?:json)?\s*(\{[\s\S]*\})\s*```"
#         r"(?:\s*\*\*Prose Explanation:\*\*\s*([\s\S]*))?"
#         r"|(\{[\s\S]*\})\s*(?:\*\*Prose Explanation:\*\*\s*([\s\S]*))?",
#         re.DOTALL
#     )

#     match = pattern.search(response_text)
#     if not match:
#         print("No JSON block found.")
#         return None, None

#     # Select the matched JSON group (fenced or unfenced)
#     json_str = (match.group(1) or match.group(3) or "").strip()
#     prose = (match.group(2) or match.group(4) or "").strip()

#     # Trim trailing junk after the last closing brace
#     last_brace = json_str.rfind("}")
#     if last_brace != -1:
#         json_str = json_str[:last_brace + 1]

#     try:
#         data = json.loads(json_str)
#     except json.JSONDecodeError as e:
#         print(f"JSON parse error: {e}")
#         print("Captured JSON preview:\n", json_str[:500])
#         data = None

#     return data, prose

In [147]:
def compare_citations(gt_citations, agent_citations, ignore_keys=('year',)):
    """
    Compare two lists of citation dicts (ground truth vs agent output).
    Removes keys in `ignore_keys` (e.g., 'year') before comparison.
    Returns (accuracy, mismatches) similar to compare_financials().
    """
    def normalize(citation):
        """Remove ignored keys and sort items for set comparison."""
        return tuple(sorted(
            (k, v) for k, v in citation.items() if k not in ignore_keys
        ))

    gt_set = {normalize(c) for c in gt_citations}
    agent_set = {normalize(c) for c in agent_citations}

    matches = gt_set & agent_set
    missing = gt_set - agent_set
    extra = agent_set - gt_set

    true_count = len(gt_set)
    agent_count = len(agent_set)

    print(f"📊 Citation Counts → Ground Truth: {true_count}, Agent: {agent_count}")
    print(f"✅ Matches: {len(matches)} / {true_count}")
    print(f"❌ Missing (in GT but not in Agent): {len(missing)}")
    print(f"⚠️ Extra (in Agent but not in GT): {len(extra)}\n")

    # Build mismatches list (labelled like in compare_financials)
    mismatches = []
    for m in missing:
        mismatches.append(("missing", dict(m)))
    for e in extra:
        mismatches.append(("extra", dict(e)))

    # Compute accuracy as % of correct citations in GT
    accuracy = (len(matches) / true_count * 100) if true_count > 0 else 0

    return accuracy, mismatches, true_count, agent_count

In [148]:
# Measure accuracy
for i, result in enumerate(agent_data):
    print(f"Q{i}:{result['question']}")
    # if not result['data'] or not result['prose']:
    #     result['data'], result['prose'] = extract_json_and_prose(result['raw_output'])
    agent_computed = result['data']['computed_values']
    gt_computed = ground_truth[i]['expected_computed_values']
    print("1) ACTUAL GT->Agent::::")
    print(gt_computed)
    print(agent_computed)

    acc, mismatches = compare_financials(agent_computed, gt_computed)
    result['answer_accuracy'] = f"{acc:.3f}"
    result['answer_mismatches'] = mismatches

    print(f"Data Value Accuracy: {acc:.1f}%")
    print("Mismatched entries:", mismatches)
    
    print("\n3) Citations GT->Agent::::")
    agent_citations = result['data']['citations']
    gt_citations =ground_truth[i]['expected_citations']
    print(gt_citations)
    print(agent_citations)
    
    cite_acc, cite_mismatches, cite_expected_count, cite_count = compare_citations(gt_citations, agent_citations)
    result['cite_accuracy'] = f"{cite_acc:.3f}"
    result['cite_mismatches'] = cite_mismatches
    result['cite_expected_count'] = cite_expected_count
    result['cite_count'] = cite_count

    print(f"Citation Value Accuracy: {cite_acc:.1f}%")
    print("Mismatched entries:", cite_mismatches)

    print("\n")


Q0:Report the Gross Margin (%) over the last 5 quarters, with values.
1) ACTUAL GT->Agent::::
[{'fiscal_year': '2025', 'fiscal_quarter': 'Q2', 'gross_margin': 75.1}, {'fiscal_year': '2025', 'fiscal_quarter': 'Q3', 'gross_margin': 74.6}, {'fiscal_year': '2025', 'fiscal_quarter': 'Q4', 'gross_margin': 73.0}, {'fiscal_year': '2026', 'fiscal_quarter': 'Q1', 'gross_margin': 60.5}, {'fiscal_year': '2026', 'fiscal_quarter': 'Q2', 'gross_margin': 72.4}]
[{'quarter': 'Q1 FY26', 'gross_margin': 60.5, 'unit': '%'}, {'quarter': 'Q4 FY25', 'gross_margin': 73.0, 'unit': '%'}, {'quarter': 'Q3 FY25', 'gross_margin': 74.6, 'unit': '%'}, {'quarter': 'Q2 FY25', 'gross_margin': 75.1, 'unit': '%'}, {'quarter': 'Q1 FY25', 'gross_margin': 78.4, 'unit': '%'}]
2) NORMALISED GT->Agent:::::::::::
{'FY2025-Q2': {'fiscal_year': '2025', 'fiscal_quarter': 'Q2', 'gross_margin': 75.1}, 'FY2025-Q3': {'fiscal_year': '2025', 'fiscal_quarter': 'Q3', 'gross_margin': 74.6}, 'FY2025-Q4': {'fiscal_year': '2025', 'fiscal_quart

In [149]:
with open('agent_result_processed.json', 'w', encoding='utf-8') as f:
    json.dump(agent_data, f, indent=2, ensure_ascii=False)

In [150]:
import json
import pandas as pd

# === 1. Load processed data ===
with open("agent_result_processed.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Ensure we have a list of questions
if not isinstance(data, list):
    raise ValueError("Expected a list of question entries.")

# === 2. Extract metrics ===
records = []
for q in data:
    qnum = q.get("question_number")
    qtext = q.get("question")
    acc_ans = float(q.get("answer_accuracy"))
    acc_cite = float(q.get("cite_accuracy"))
    cite_expected_count = q.get("cite_expected_count")
    cite_count = q.get("cite_count")

    timing = q.get("callback_timing", {})
    T_reason = timing.get("T_reason")
    T_generate = timing.get("T_generate")
    T_total = timing.get("T_total")

    # Derived metric: reasoning without generation
    T_reason_only = T_reason - T_generate

    prompt_tokens = timing.get("prompt_tokens")
    completion_tokens = timing.get("completion_tokens")
    total_tokens = timing.get("total_tokens")
    num_llm_calls = timing.get("num_llm_calls")

    tool_list = q.get("data", {}).get("tools")
    tool_count = q.get("data", {}).get("tools_count")
    tools_used = ", ".join(tool_list)

    records.append({
        "Question #": qnum,
        "Question": qtext,
        "Answer Accuracy (%)": acc_ans,
        "Citation Accuracy (%)": acc_cite,
        "Citation Count": cite_count,
        "Expected Citation Count": cite_expected_count,
        "Tools Used": tools_used,
        "Tool Count": tool_count,
        # "Reasoning Time (s)": T_reason,
        "Generation Time (s)": T_generate,
        "Reasoning w/o Gen (s)": T_reason_only,
        "Total Time (s)": T_total,
        "Prompt Tokens": prompt_tokens,
        "Completion Tokens": completion_tokens,
        "Total Tokens": total_tokens,
        "LLM Calls": num_llm_calls
    })

df = pd.DataFrame(records)
df

,Question #,Question,Answer Accuracy (%),Citation Accuracy (%),Citation Count,Expected Citation Count,Tools Used,Tool Count,Generation Time (s),Reasoning w/o Gen (s),Total Time (s),Prompt Tokens,Completion Tokens,Total Tokens,LLM Calls
0,1,Report the Gross Margin (%) over the last 5 qu...,80.0,80.0,5,5,retriever,1,7.8227,2.4797,10.8420,4450,847,5297,2
1,2,Show Operating Expenses for the last 3 fiscal ...,50.0,0.0,3,1,"retriever, comparison_tool",2,4.8197,4.6477,10.4201,8587,806,9393,3
2,3,Calculate the Operating Efficiency Ratio (Opex...,0.0,0.0,1,1,"retriever, calculator_tool",2,5.4767,4.8229,13.9649,8512,882,9394,3


In [151]:
for q in data:
    print("==================================================================Q==================================================================")
    print(q.get("question"))
    print(q.get("prose"))
    print("\n\n")

==================================================================Q==================================================================
Report the Gross Margin (%) over the last 5 quarters, with values.
| Quarter  | Gross Margin (%) | Source Report                          | Page | Section                      |
|----------|------------------|----------------------------------------|------|------------------------------|
| Q1 FY26  | 60.5%            | FY26Q1_QuarterlyPresentation           | 6    | Q1 FY26 Financial Summary    |
| Q4 FY25  | 73.0%            | FY25Q4_QuarterlyPresentation           | 6    | Q4 FY25 Financial Summary    |
| Q3 FY25  | 74.6%            | FY25Q3_QuarterlyPresentation           | 6    | Q3 FY25 Financial Summary    |
| Q2 FY25  | 75.1%            | FY25Q2_QuarterlyPresentation           | 6    | Q2 FY25 Financial Summary    |
| Q1 FY25  | 78.4%            | FY25Q1_10Q                             | 27   | Item 1A. Risk                |

The table above shows